# Multi-head Attention
La mécanisme d'attention multi-têtes est un composant clé de l'architecture Transformer utilisée pour les tâches de traitement du langage naturel (NLP). Il permet au modèle de se concentrer sur différentes parties de la séquence d'entrée simultanément, lui offrant ainsi la capacité de capturer divers types de relations entre les mots.

Dans le contexte du NLP, supposons que nous ayons une séquence d'incorporations d'entrée $X = \{x_1, x_2, \ldots, x_n\}$, où $x_i$ représente l'incorporation du $i$-ème mot dans la séquence. Le mécanisme d'attention multi-têtes fonctionne comme suit :

1. **Transformations Linéaires** : Les incorporations d'entrée sont transformées linéairement en trois ensembles différents d'incorporations - Requête ($Q$), Clé ($K$), et Valeur ($V$).

   $$
   Q = XW_Q, \quad K = XW_K, \quad V = XW_V
   $$

   Ici, $W_Q$, $W_K$ et $W_V$ sont des matrices de poids apprenables.

2. **Division en Plusieurs Têtes** : Chacune des matrices $Q$, $K$ et $V$ est divisée en plusieurs matrices plus petites (têtes). Supposons que nous ayons $h$ têtes, les matrices divisées deviennent :

   $$
   Q_i = QW_{Qi}, \quad K_i = KW_{Ki}, \quad V_i = VW_{Vi}
   $$

   où $W_{Qi}$, $W_{Ki}$ et $W_{Vi}$ sont des matrices de poids spécifiques à la $i$-ème tête.

3. **Attention à Produit Scalaire Normalisé** : Pour chaque tête, les scores d'attention sont calculés en utilisant un mécanisme d'attention à produit scalaire normalisé :

   $$
   \text{Attention}(Q_i, K_i, V_i) = \text{softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_k}}\right) V_i
   $$

   où $d_k$ est la dimension des vecteurs clés.

4. **Concaténation et Transformation Linéaire** : Les résultats de toutes les têtes sont concaténés et transformés linéairement pour obtenir la sortie finale de l'attention multi-têtes :

   $$
   \text{MultiTête}(Q, K, V) = \text{Concat}(\text{Attention}_1, \ldots, \text{Attention}_h)W_O
   $$

   où $W_O$ est une matrice de poids apprenable.

5. **Connexion Résiduelle et Normalisation de Couche** : La sortie de l'attention multi-têtes est ajoutée à l'entrée d'origine (avec une connexion directe) puis soumise à une opération de normalisation de couche.

   $$
   \text{Sortie} = \text{LayerNorm}(X + \text{MultiTête}(Q, K, V))
   $$

Le modèle Transformer répète généralement ces étapes plusieurs fois dans une couche, en plus de réseaux neuronaux à propagation avant et d'opérations de normalisation supplémentaires.

Ce mécanisme d'attention multi-têtes permet au modèle de se concentrer sur différentes parties de la séquence d'entrée avec différentes projections linéaires apprises, lui permettant de capturer différents types d'informations et de relations dans les données.

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math

In [ ]:
def Attention(q, k, v, dh, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2,-1))/math.sqrt(dh)
    if mask is not None:
        mask = mask.unsqueeze(1);
        scores = scores.masked_fill(mask == 0, -1e9)

    scores = torch.softmax(scores, dim=-1)
    if dropout is not None:
        scores = dropout(scores)
    output = torch.matmul(scores, v)
    return output,scores

In [ ]:
class MHAttention(nn.Module):
    def __init__(self, d_model, n_heads, droprate=0.1):
        super(MHAttention, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.dh = d_model//n_heads
        # couches d'apprentissage pour q,k,v
        self.q_matrix = nn.Linear(d_model, d_model)
        self.k_matrix = nn.Linear(d_model, d_model)
        self.v_matrix = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(droprate)
        self.fc1 = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # input = batch_size X seq_len X d_model into batch_size X heads X seq_len X d_model/heads
        q = self.q_matrix(q); q = q.view(q.size(0), self.n_heads, -1, self.dh)
        k = self.k_matrix(k); k = k.view(k.size(0), self.n_heads, -1, self.dh)
        v = self.v_matrix(v); v = v.view(v.size(0), self.n_heads, -1, self.dh)
        scores,attention_weights = Attention(q, k, v, self.dh, mask, self.dropout)
        scores = scores.reshape(q.size(0), -1, self.d_model)
        output = self.fc1(scores)
        return output,attention_weights


In [ ]:
batch_size = 1
seq_len = 5
d_model = 64
num_heads = 4

embedding  = torch.randn(batch_size, seq_len, d_model)

In [ ]:
# Instanciation de la couche d'attention à plusieurs têtes
multi_head_attention = MHAttention(d_model, num_heads)

# Appliquer l'attention à plusieurs têtes
output, attention_weights = multi_head_attention(embedding,embedding,embedding)

print("Output:", output.shape)
print("Attentions:", attention_weights.shape)

Output: torch.Size([1, 5, 64])
Attentions: torch.Size([1, 4, 5, 5])


# Masked multi-head attention
Le masque utilisé dans l'attention multi-têtes masquée est une matrice binaire qui empêche certaines positions d'être attentives à d'autres positions dans la séquence. Plus précisément, le masque garantit qu'une position donnée ne peut s'intéresser qu'aux positions qui la précèdent.

Par exemple, dans une tâche de modélisation du langage où le modèle est entraîné à prédire le mot suivant dans une phrase, il est important que le modèle n'ait pas accès aux mots futurs pendant l'entraînement. C'est là que le masquage entre en jeu.

Le masque est généralement une matrice triangulaire inférieure, dans laquelle les éléments situés au-dessus de la diagonale principale sont fixés à une grande valeur négative (généralement `-inf`), et les éléments situés au-dessous ou sur la diagonale principale sont fixés à zéro. Cela garantit que lorsque le masque est ajouté aux scores d'attention, l'opération softmax ignorera effectivement les positions masquées, car l'exponentialisation de grandes valeurs négatives aboutira à des valeurs proches de zéro.



In [ ]:


def generate_mask(seq_len):
    mask = (torch.triu(torch.ones(seq_len, seq_len)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

seq_len = 5
mask = generate_mask(seq_len)

print("Mask:")
print(mask)


Mask:
tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])
